<a href="https://colab.research.google.com/github/ronglu-stanford/RL_reference_public/blob/main/3_gpu_computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numba import jit
import random
import timeit
import numpy as np
import math

# Numba JIT compiled Python code

In [ ]:
@jit(nopython=True)
def monte_carlo_pi_fast(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples    

In [ ]:
num_runs = 4

nsamples = 1000000

duration = timeit.Timer(lambda: monte_carlo_pi_fast(nsamples)).timeit(number = num_runs)
print(f'On average it took {duration/num_runs} seconds')

duration = timeit.Timer(lambda: monte_carlo_pi(nsamples)).timeit(number = num_runs)
print(f'On average it took {duration/num_runs} seconds')

On average it took 0.036436872500019035 seconds
On average it took 0.3770847377500104 seconds


# Numba, generating CUDA code

In [ ]:
from numba import cuda, float32

@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [ ]:
shp = (1000,1000)
A = np.random.random(shp)
B = np.random.random(shp)
C = np.zeros(shp)

threadsperblock = (16, 16)
blockspergrid_x = math.ceil(C.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(C.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

duration = timeit.Timer(lambda: matmul[blockspergrid, threadsperblock](A, B, C)).timeit(number = 1)
print(f'On average it took {duration} seconds')
# You will need a large matrix to see a speed-up
# The running time includes transferring data to and from the GPU

duration = timeit.Timer(lambda: A.dot(B)).timeit(number = 1)
print(f'On average it took {duration} seconds')

print(np.linalg.norm(C - A.dot(B)))

On average it took 0.25407473700010996 seconds
On average it took 0.07713242299996637 seconds
2.0377246600389243e-10


# OpenMP with distribute parallel for

In [ ]:
%%file omp_distribute.cpp

#include <cassert>
#include <cstdio>
#include <vector>

using namespace std;

int main(void) {
  const int N = 1 << 21;
  printf("N = %d\n", N);

  float *A = new float[N];
  float *B = new float[N];
  float *C = new float[N];

  for (int i = 0; i < N; ++i) {
    A[i] = 0;
    B[i] = i;
    C[i] = 3 * i;
  }

  int nteams = 1024;
  int block_threads = N / nteams;
  const int k = 4;
#pragma omp target map(tofrom : A) map(to : B, C)
#pragma omp teams num_teams(nteams)
#pragma omp distribute parallel for dist_schedule(static, block_threads)
  for (int i = 0; i < N; ++i) {
    A[i] = B[i] + k * C[i];
  }

  for (int i = 0; i < N; ++i) {
    assert(A[i] == i + 3 * k * i);
  }

  printf("PASS\n");
  return 0;
}

Overwriting omp_distribute.cpp


In [ ]:
# This will run on the CPU not the GPU
# g++ needs to be configured properly to generate CUDA code
# see https://gcc.gnu.org/wiki/Offloading
# https://www.openmp.org/resources/openmp-compilers-tools/
!name=omp_distribute && g++ -std=c++11 -o $name $name.cpp -fopenmp && ./$name

N = 2097152
PASS


# Example CUDA code, how to add two matrices

In [ ]:
%%file matrix_add.cu

#include <iostream>
#include <vector>
#include <cstdlib>
#include <cstdio>
#include <cassert>
#include <unistd.h>

using std::vector;

__global__
void Initialize(int n, int* a, int* b) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if(i < n && j < n) {
        a[n*i + j] = j;
        b[n*i + j] = i-2*j;
    }
}

__global__
void Add(int n, int* a, int* b, int* c) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;
    if(i < n && j < n) {
        c[n*i + j] = a[n*i + j] + b[n*i + j];
    }
}

int main(int argc, const char** argv) {

    int n = 1024;
    int n_thread = 512;

    printf("Using %d threads = %d warps\n",n_thread, (n_thread+31)/32);   
    printf("Dimensions of matrix: %5d x %5d\n",n,n);

    int* d_a, *d_b, *d_c;

    /* Allocate memory */
    cudaMalloc(&d_a, sizeof(int) * n*n);
    cudaMalloc(&d_b, sizeof(int) * n*n);
    cudaMalloc(&d_c, sizeof(int) * n*n);

    dim3 th_block(32,n_thread/32);
    int blocks_per_grid_x = (n + th_block.x - 1) / th_block.x;
    int blocks_per_grid_y = (n + th_block.y - 1) / th_block.y;
    /* This formula is needed to make sure we process all entries in matrix */
    dim3 num_blocks(blocks_per_grid_x, blocks_per_grid_y);

    printf("Dimension of thread block: %2d x %2d\n", th_block.x, th_block.y);
    printf("Dimension of grid: %3d x %3d\n", num_blocks.x, num_blocks.y);

    /* Run calculation on GPU */

    /* Initialize matrices */
    Initialize<<<num_blocks, th_block>>>(n, d_a, d_b);

    /* C = A + B */
    Add<<<num_blocks, th_block>>>(n, d_a, d_b, d_c);

    cudaFree(d_a); 
    cudaFree(d_b);    

    /* Note that kernels execute asynchronously.
       They will fail without any error message!
       This can be confusing when debugging.
       The output arrays will be left uninitialized with no warning.
       */

    vector<int> h_c(n*n);
    /* Copy the result back to the CPU */
    cudaMemcpy(&h_c[0], d_c, sizeof(int) * n*n, cudaMemcpyDeviceToHost);

    cudaFree(d_c);     

    /* Test result */
    for(int i = 0; i < n; ++i) {
        for(int j = 0; j < n; ++j) {
            if(!(h_c[n*i + j] == i-j)) {
                printf("%d %d %d %d %d\n",n,i,j,h_c[n*i + j],i-j);
            }
            assert(h_c[n*i + j] == i-j);
        }
    }         

    printf("All tests have passed; calculation is correct.\n");

    return 0;
}


Overwriting matrix_add.cu


In [ ]:
!name=matrix_add && nvcc -o $name $name.cu && ./$name

Using 512 threads = 16 warps
Dimensions of matrix:  1024 x  1024
Dimension of thread block: 32 x 16
Dimension of grid:  32 x  64
All tests have passed; calculation is correct.


# Exercise: n-body problem using CUDA

In [ ]:
%%file nbody.cu

#include <cassert>
#include <cmath>
#include <cstdio>
#include <cstdlib>
#include <iostream>
#include <vector>

using namespace std;

// Pseudo-random number generator
__host__ __device__
int safe_rand(unsigned int *seed) {
    unsigned int next = *seed;
    int result;
    
    next *= 1103515245;
    next += 12345;
    result = (unsigned int)(next / 65536) % 2048;
    
    next *= 1103515245;
    next += 12345;
    result <<= 10;
    result ^= (unsigned int)(next / 65536) % 1024;
    
    next *= 1103515245;
    next += 12345;
    result <<= 10;
    result ^= (unsigned int)(next / 65536) % 1024;
    
    *seed = next;
    
    return result;
}

__host__ __device__
float force(const float x) { return -2. * atan(x) / (x * x + 1.); }

__global__
void initialize(int n, float * __restrict__ x) {
  // fill-in the code
  // Use code from shared_memory.ipynb :
  // unsigned int seed = 4357U + unsigned(i) * 1103515245;
  // x[i] = float(safe_rand(&seed)) / RAND_MAX;
}

__global__
void compute_force(int n, float * __restrict__ x, float * __restrict__ f) {
  // fill-in the code
}

int main(void) {
    const unsigned n = 10000;
    const unsigned n_thread = 256;
    
    const unsigned n_bytes = sizeof(float) * n;

    float* d_x, *d_f;
    
    /* Allocate memory and initialize */
    cudaMalloc(&d_x, n_bytes);
    cudaMalloc(&d_f, n_bytes);
    cudaMemset(d_f, 0, n_bytes);
    
    dim3 th_block(n_thread);
    // fill-in the code
    // const unsigned blocks_per_grid_x = ...;
    // dim3 num_blocks(...);    
    
    // fill-in the code
    // initialize<<<..., ...>>>(...);
    // compute_force<<<..., ...>>>(...);  
    
    cudaFree(d_x);
    
    /* Copy the result back to the CPU */
    vector<float> h_f(n);
    cudaMemcpy(&h_f[0], d_f, n_bytes, cudaMemcpyDeviceToHost);      

    cudaFree(d_f);     
    
    // Test
    vector<float> x(n);
    {
        for (int i = 0; i < n; ++i) {
            unsigned int seed = 4357U + unsigned(i) * 1103515245;        
            x[i] = float(safe_rand(&seed)) / RAND_MAX;
            assert(x[i] > 0 && x[i] < 1);
        }
    }
    
    vector<float> f(n,0);
    
    for (int i = 0; i < n; ++i) {
        for (int j = i + 1; j < n; ++j) {
            const float x_ = x[i] - x[j];
            const float f_ = force(x_);
            f[i] += f_;
            f[j] -= f_;
        }
    }
    
    {        
        float max_err = 0;
        float scale = 0;
        for (int i = 0; i < n; ++i) {
            max_err = max(max_err, fabs(h_f[i] - f[i]));
            scale = max(scale, fabs(f[i]));
        }
        printf("largest error: %g\n", max_err/scale);        
     
        for (int i = 0; i < n; ++i) assert(fabs(h_f[i] - f[i]) < 1e-5*scale);        
        cout << "PASS\n";
    }
    
    return 0;
}

Overwriting nbody.cu


In [ ]:
!name=nbody && nvcc -o $name $name.cu && ./$name

nbody.cu(65): warning: variable "th_block" was declared but never referenced

largest error: 1
nbody: nbody.cu:112: int main(): Assertion `fabs(h_f[i] - f[i]) < 1e-5*scale' failed.


Complete the exercise above before looking at the solution!!

# Solution

## n-body problem using CUDA

In [ ]:
%%file nbody.cu

#include <cassert>
#include <cmath>
#include <cstdio>
#include <cstdlib>
#include <iostream>
#include <vector>

using namespace std;

// Pseudo-random number generator
__host__ __device__
int safe_rand(unsigned int *seed) {
    unsigned int next = *seed;
    int result;
    
    next *= 1103515245;
    next += 12345;
    result = (unsigned int)(next / 65536) % 2048;
    
    next *= 1103515245;
    next += 12345;
    result <<= 10;
    result ^= (unsigned int)(next / 65536) % 1024;
    
    next *= 1103515245;
    next += 12345;
    result <<= 10;
    result ^= (unsigned int)(next / 65536) % 1024;
    
    *seed = next;
    
    return result;
}

__host__ __device__
float force(const float x) { return -2. * atan(x) / (x * x + 1.); }

__global__
void initialize(int n, float * __restrict__ x) {
    long i = blockIdx.x * blockDim.x + threadIdx.x;
    
    if(i < n) {
        unsigned int seed = 4357U + unsigned(i) * 1103515245;
        x[i] = float(safe_rand(&seed)) / RAND_MAX;
    }
}

__global__
void compute_force(int n, float * __restrict__ x, float * __restrict__ f) {
    long i = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (i < n) {
        for (int j = 0; j < n; ++j) {
            if (j != i) {
                const float x_ = x[i] - x[j];
                const float f_ = force(x_);
                f[i] += f_; 
                // there is no race condition on += in this algorithm
            }
        }
    }
}

int main(void) {
    const unsigned n = 10000;
    const unsigned n_thread = 256;
    
    const unsigned n_bytes = sizeof(float) * n;

    float* d_x, *d_f;
    
    /* Allocate memory and initialize */
    cudaMalloc(&d_x, n_bytes);
    cudaMalloc(&d_f, n_bytes);
    cudaMemset(d_f, 0, n_bytes);
    
    dim3 th_block(n_thread);
    const unsigned blocks_per_grid_x = (n + th_block.x - 1) / th_block.x;
    dim3 num_blocks(blocks_per_grid_x);    
    
    initialize<<<num_blocks, th_block>>>(n, d_x);
    compute_force<<<num_blocks, th_block>>>(n, d_x, d_f);  

    cudaFree(d_x);
    
    /* Copy the result back to the CPU */
    vector<float> h_f(n);
    cudaMemcpy(&h_f[0], d_f, n_bytes, cudaMemcpyDeviceToHost);      

    cudaFree(d_f);    
    
    // Test
    vector<float> x(n);
    {
        for (int i = 0; i < n; ++i) {
            unsigned int seed = 4357U + unsigned(i) * 1103515245;        
            x[i] = float(safe_rand(&seed)) / RAND_MAX;
            assert(x[i] > 0 && x[i] < 1);
        }
    }
    
    vector<float> f(n,0);
    
    for (int i = 0; i < n; ++i) {
        for (int j = i + 1; j < n; ++j) {
            const float x_ = x[i] - x[j];
            const float f_ = force(x_);
            f[i] += f_;
            f[j] -= f_;
        }
    }
    
    {        
        float max_err = 0;
        float scale = 0;
        for (int i = 0; i < n; ++i) {
            max_err = max(max_err, fabs(h_f[i] - f[i]));
            scale = max(scale, fabs(f[i]));
        }
        printf("largest error: %g\n", max_err/scale);        
     
        for (int i = 0; i < n; ++i) assert(fabs(h_f[i] - f[i]) < 1e-5*scale);        
        cout << "PASS\n";
    }    
    
    return 0;
}

Overwriting nbody.cu


In [ ]:
!name=nbody && nvcc -o $name $name.cu && ./$name

largest error: 2.37414e-07
PASS
